In [62]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [100]:
stock_data = pd.read_csv('datasets/ABEV3.csv')
stock_data.shape, 'NaN: {}'.format(stock_data.isnull().sum().sum())

((4971, 5), 'NaN: 0')

Prepara base de dados para treinamento e teste

In [109]:
lenght_test = (stock_data.shape[0] * 0.3)
position = int((stock_data.shape[0]-lenght_test))

stock_train = stock_data.iloc[0:position].copy()
stock_train.reset_index(drop=True, inplace=True)

stock_test = stock_data.iloc[position:].copy()
stock_test.reset_index(drop=True, inplace=True)

In [110]:
# price Open
price_open_train  = stock_train.iloc[:, 1:2].values
price_open_train

array([[ 7.7958],
       [ 7.7383],
       [ 7.7476],
       ...,
       [16.32  ],
       [16.29  ],
       [16.23  ]])

In [112]:
# normalization data
normalizer = MinMaxScaler(feature_range=(0,1))
price_open_train_normalizer = normalizer.fit_transform(price_open_train)
price_open_train_normalizer

array([[0.31583513],
       [0.31208505],
       [0.31269158],
       ...,
       [0.8717733 ],
       [0.86981674],
       [0.86590361]])

In [113]:
lenght = stock_train.shape[0]
dsstart = 90

predictors=[]
target_price=[]

for i in range(dsstart, lenght):
    predictors.append(price_open_train_normalizer[i-dsstart:i])
    target_price.append(price_open_train_normalizer[i])
    
predictors = np.array(predictors)
target_price = np.array(target_price)

In [114]:
# organizando a estrutura para o Keras
predictors = np.reshape(predictors, (target_price.shape[0], predictors.shape[1], 1))

In [115]:
regressor = Sequential()
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(predictors.shape[1], 1)))
regressor.add(Dropout(0.3))# dropar 30% dos dados para evitar overfitting

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units=1, activation='linear'))
regressor.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
regressor.fit(predictors, target_price, epochs=100, batch_size=32)

Epoch 1/100
 46/106 [============>.................] - ETA: 5s - loss: 0.0219 - mean_absolute_error: 0.0939

# Predict

In [242]:
base_teste = pd.read_csv('datasets/petr4_teste.csv')
preco_real_teste = base_teste.iloc[:, 1:2].values
base_teste.shape

(22, 7)

In [243]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)
base_completa.shape

(1264,)

In [244]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values
entradas = entradas.reshape(-1, 1)
entradas = normalizer.transform(entradas)
entradas.shape

(112, 1)

In [245]:
X_teste=[]
for i in range(dsstart, entradas.shape[0]):
    X_teste.append(entradas[i-dsstart:i])
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
X_teste.shape

(22, 90, 1)

In [246]:
previsoes = regressor.predict(X_teste)

In [247]:
previsoes = normalizer.inverse_transform(previsoes)

In [248]:
len(preco_real_teste), len(previsoes)

(22, 22)

In [257]:
previsoes.mean(), preco_real_teste.mean(), (abs(previsoes.mean()-preco_real_teste.mean()))

(17.908348, 17.87454563636364, 0.03380244713245517)

In [258]:
plt.figure(figsize=(15,6))
preco_real_teste.plot(color='red', label='Preço Real')
plt.plot(previsoes, color='blue', label='Previsões')
plt.grid(True)
plt.legend()

AttributeError: 'numpy.ndarray' object has no attribute 'plot'

<Figure size 1080x432 with 0 Axes>